In [36]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import pandas as pd
import os
from dotenv import load_dotenv

In [37]:
# Load the OpenAI API key from the environment variable
load_dotenv()
open_api_key = os.getenv("OPENAI_API_KEY")

In [38]:
# Load the documents
# Make sure the file path is correct
loader = CSVLoader(file_path='data/Hiker_Food.csv')

In [39]:
index_creator = VectorstoreIndexCreator()
docsearch = index_creator.from_loaders([loader])

In [48]:
# Make sure to replace "stuff" with a valid chain type if needed
chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(streaming=True), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question", return_source_documents=True)

In [49]:
query = "How many calories per oz are in a kind bar?"
response = chain({"question": query})
print(response['result'])

There are 128 calories per ounce in a Kind bar.


In [50]:
def summarize_response(response):
    result = response['result']
    source_docs = response['source_documents']
    
    summary = f"Result: {result}\n\nSource Documents:\n\n"
    for doc in source_docs:
        page_content = doc.page_content.split('\n')
        brand_line = [line for line in page_content if "Brand:" in line][0]
        bold_brand_line = f"**{brand_line}**"  # Markdown syntax for bold
        other_lines = [line for line in page_content if "Brand:" not in line]
        table = bold_brand_line + " | " + " | ".join(other_lines)
        summary += " - " + table + "\n\n"
    
    return summary


In [51]:
query = "How many calories per oz are in a kind bar?"
response = chain({"question": query})
summary = summarize_response(response)
print(summary)

Result: A Kind bar has 128 calories per ounce.

Source Documents:

 - **Brand: Kind** | Flavor: Fruit & Nut | Class: Bar | Serv(oz): 1.41 | Serv(g): 40 | Cal/Serv: 180 | Fat: 13 | Na: 15 | K: 180 | Carbs: 17 | Fiber: 3 | Sugar: 7 | Other: 7 | Protein: 6 | kcal/g: 4.50 | kcal/oz: 128 | Carb/Pro: 2.8 | Fat %: 56% | Sugar %: 13% | Na/kcal: 0.08

 - **Brand: Kind** | Flavor: Fruit & Nut | Class: Bar | Serv(oz): 1.41 | Serv(g): 40 | Cal/Serv: 180 | Fat: 13 | Na: 15 | K: 180 | Carbs: 17 | Fiber: 3 | Sugar: 7 | Other: 7 | Protein: 6 | kcal/g: 4.50 | kcal/oz: 128 | Carb/Pro: 2.8 | Fat %: 56% | Sugar %: 13% | Na/kcal: 0.08

 - **Brand: Kind (Protein)** | Flavor: Caramel Nut | Class: Bar | Serv(oz): 1.76 | Serv(g): 50 | Cal/Serv: 240 | Fat: 17 | Na: 75 | K:  | Carbs: 18 | Fiber: 5 | Sugar: 8 | Other: 5 | Protein: 12 | kcal/g: 4.80 | kcal/oz: 136 | Carb/Pro: 1.5 | Fat %: 56% | Sugar %: 12% | Na/kcal: 0.31

 - **Brand: Kind (Protein)** | Flavor: Caramel Nut | Class: Bar | Serv(oz): 1.76 | Serv(g):

In [52]:
query = "If I like Peanut Butter M&Ms, what other snacks should i consider if want similar nutrients but with less sugar?"
response = chain({"question": query})
summary = summarize_response(response)
print(summary)

Result: If you like Peanut Butter M&Ms and want similar nutrients but with less sugar, you may consider the regular Peanut M&Ms. They have similar fat, calorie, and protein content but contain less sugar compared to the Peanut Butter M&Ms.

Source Documents:

 - **Brand: M&Ms** | Flavor: Peanut Butter | Class: Candy | Serv(oz): 0.99 | Serv(g): 28 | Cal/Serv: 150 | Fat: 8 | Na: 60 | K:  | Carbs: 16 | Fiber: 1 | Sugar: 14 | Other: 1 | Protein: 3 | kcal/g: 5.36 | kcal/oz: 152 | Carb/Pro: 5.3 | Fat %: 49% | Sugar %: 38% | Na/kcal: 0.40

 - **Brand: M&Ms** | Flavor: Peanut Butter | Class: Candy | Serv(oz): 0.99 | Serv(g): 28 | Cal/Serv: 150 | Fat: 8 | Na: 60 | K:  | Carbs: 16 | Fiber: 1 | Sugar: 14 | Other: 1 | Protein: 3 | kcal/g: 5.36 | kcal/oz: 152 | Carb/Pro: 5.3 | Fat %: 49% | Sugar %: 38% | Na/kcal: 0.40

 - **Brand: M&Ms** | Flavor: Peanut | Class: Candy | Serv(oz): 1.48 | Serv(g): 42 | Cal/Serv: 220 | Fat: 11 | Na: 20 | K:  | Carbs: 26 | Fiber: 1 | Sugar: 22 | Other: 3 | Protein: 4 

In [53]:
query = "What are the highest calorie per oz foods?"
response = chain({"question": query})
summary = summarize_response(response)
print(summary)

Result: The highest calorie per oz food is Jelly Belly Extreme Sport Beans, Watermelon with 101 calories per oz.

Source Documents:

 - **Brand: Jelly Belly** | Flavor: Extreme Sport Beans, Watermelon | Class: Candy | Serv(oz): 0.99 | Serv(g): 28 | Cal/Serv: 100 | Fat: 0 | Na: 80 | K: 40 | Carbs: 25 | Fiber:  | Sugar: 17 | Other: 8 | Protein: 0 | kcal/g: 3.57 | kcal/oz: 101 | Carb/Pro: #DIV/0! | Fat %: 0% | Sugar %: 68% | Na/kcal: 0.80

 - **Brand: Jelly Belly** | Flavor: Extreme Sport Beans, Watermelon | Class: Candy | Serv(oz): 0.99 | Serv(g): 28 | Cal/Serv: 100 | Fat: 0 | Na: 80 | K: 40 | Carbs: 25 | Fiber:  | Sugar: 17 | Other: 8 | Protein: 0 | kcal/g: 3.57 | kcal/oz: 101 | Carb/Pro: #DIV/0! | Fat %: 0% | Sugar %: 68% | Na/kcal: 0.80

 - **Brand: Jelly Belly** | Flavor: Jelly Beans, assorted flavors | Class: Candy | Serv(oz): 1.41 | Serv(g): 40 | Cal/Serv: 140 | Fat: 0 | Na: 15 | K:  | Carbs: 37 | Fiber: 0 | Sugar: 28 | Other: 9 | Protein: 0 | kcal/g: 3.50 | kcal/oz: 99 | Carb/Pro: